In [1]:
%%capture pipoutput
%pip install trino python-dotenv
%pip install --upgrade sqlalchemy sqlalchemy-trino
%pip install osc-ingest-tools

In [2]:
import osc_ingest_trino as osc
osc.load_credentials_dotenv()

# my default personal identity: has full "admin" privileges
engine_eje = osc.attach_trino_engine()

# user1 has no admin privs but is declared on a few demo groups
engine_user1 = osc.attach_trino_engine('TRINO_OSCU1')

# user6 has no group memberships or any other declared privileges
engine_user6 = osc.attach_trino_engine('TRINO_OSCU6')

/opt/app-root/lib64/python3.8/site-packages/sqlalchemy_trino/dialect.py:259: SAWarning: Dialect trino:rest will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  res = connection.execute(sql.text(query)).scalar()


In [3]:
%xmode Minimal
# just report raw exceptions in output - makes this story easier to read

Exception reporting mode: Minimal


In [4]:
sql = """
select current_groups()
"""
qres = engine_eje.execute(sql)
print(f'eje:  {qres.fetchall()}')
qres = engine_user1.execute(sql)
print(f'user1:  {qres.fetchall()}')
qres = engine_user6.execute(sql)
print(f'user6:  {qres.fetchall()}')

eje:  [(['aicoe_osc_demo', 'admins'],)]
user1:  [(['demo_dv_dev', 'demo_dv_eval', 'demo_dv_licensed'],)]
user6:  [([],)]


In [5]:
sql = """
show catalogs
"""
qres = engine_eje.execute(sql)
print(f'eje:  {qres.fetchall()}')
qres = engine_user1.execute(sql)
print(f'user1:  {qres.fetchall()}')
qres = engine_user6.execute(sql)
print(f'user6:  {qres.fetchall()}')

eje:  [('jmx',), ('osc_datacommons_dev',), ('osc_datacommons_iceberg_dev',), ('osc_datacommons_prod',), ('system',)]
user1:  [('osc_datacommons_dev',)]
user6:  []


In [6]:
qres = engine_eje.execute("show schemas in osc_datacommons_dev")
qres.fetchall()[:5]

[('aicoe_osc_demo',),
 ('company_data',),
 ('default',),
 ('defaultschema1',),
 ('demo',)]

In [7]:
qres = engine_user1.execute("show schemas in osc_datacommons_dev")
qres.fetchall()[:5]

[('aicoe_osc_demo',),
 ('company_data',),
 ('default',),
 ('defaultschema1',),
 ('demo',)]

In [8]:
qres = engine_user6.execute("show schemas in osc_datacommons_dev")
qres.fetchall()[:5]

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot access catalog osc_datacommons_dev", query_id=20220216_234016_00013_abma7)

In [9]:
qres = engine_eje.execute("show tables in osc_datacommons_dev.gleif")
qres.fetchall()

[('gleif_direct_issuer_ultimate_issuer',), ('gleif_isin_lei',)]

In [10]:
qres = engine_user1.execute("show tables in osc_datacommons_dev.gleif")
qres.fetchall()

[]

In [11]:
qres = engine_user6.execute("show tables in osc_datacommons_dev.gleif")
qres.fetchall()

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot access catalog osc_datacommons_dev", query_id=20220216_234024_00016_abma7)

In [12]:
qres = engine_eje.execute("select * from osc_datacommons_dev.gleif.gleif_isin_lei limit 5")
qres.fetchall()

[('001GPB6A9XPE8XJICC14', 'US3158052262', '2021-12-20 08:00:00.000'),
 ('00EHHQ2ZHDCFXJCPCL46', 'US92204Q1031', '2021-12-20 08:00:00.000'),
 ('00KLB2PFTM3060S2N216', 'US4138382027', '2021-12-20 08:00:00.000'),
 ('00KLB2PFTM3060S2N216', 'US4138385749', '2021-12-20 08:00:00.000'),
 ('01ERPZV3DOLNXY2MLB90', 'US531554AA10', '2021-12-20 08:00:00.000')]

In [13]:
qres = engine_user1.execute("select * from osc_datacommons_dev.gleif.gleif_isin_lei limit 5")
qres.fetchall()

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot select from table osc_datacommons_dev.gleif.gleif_isin_lei", query_id=20220216_234033_00018_abma7)

In [14]:
qres = engine_user6.execute("select * from osc_datacommons_dev.gleif.gleif_isin_lei limit 5")
qres.fetchall()

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot access catalog osc_datacommons_dev", query_id=20220216_234049_00019_abma7)